In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm

from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib


In [2]:
#Q1_TRAINING_DATA_FILE = 'q1_train.npy'
#Q2_TRAINING_DATA_FILE = 'q2_train.npy'
#LABEL_TRAINING_DATA_FILE = 'label_train.npy'
#WORD_EMBEDDING_MATRIX_FILE = 'word_embedding_matrix.npy'
#NB_WORDS_DATA_FILE = 'nb_words.json'
#MODEL_WEIGHTS_FILE = 'question_pairs_weights.h5'
MAX_SEQUENCE_LENGTH = 30
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 0.1
#RNG_SEED = 13371447
NB_EPOCHS = 500

In [3]:
from gensim.models import KeyedVectors

In [4]:
word2vec = KeyedVectors.load_word2vec_format('../tmp/GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
MAX_NB_WORDS = 200000
EMBEDDING_DIM = 300

from numpy import linalg as LA
LA.norm(word2vec.word_vec('yes') - word2vec.word_vec('no'))

2.4503

In [6]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten
from keras.layers.wrappers import TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint
from keras import backend as K

Using TensorFlow backend.


In [7]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [8]:
X_train = pd.read_csv("data/train_inter.csv")
X_test = pd.read_csv("data/test_inter.csv")

In [9]:
X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))
X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

#X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
#X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())
#X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
#X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [ ]:
del X_test

In [10]:
import gc
gc.collect()

21

In [11]:
train_texts1 = []
train_texts2 = []
for i in X_train.question1:
    train_texts1.append(i)
for i in X_train.question2:
    train_texts2.append(i)
    
test_texts1 = []
test_texts2 = []
for i in X_test.question1:
    test_texts1.append(i)
for i in X_test.question2:
    test_texts2.append(i)

In [12]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train_texts1 + train_texts2 + test_texts1 + test_texts2)

In [13]:
train_sequences_1 = tokenizer.texts_to_sequences(train_texts1)
train_sequences_2 = tokenizer.texts_to_sequences(train_texts2)
test_sequences_1 = tokenizer.texts_to_sequences(test_texts1)
test_sequences_2 = tokenizer.texts_to_sequences(test_texts2)


train_sequences_1 = pad_sequences(train_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
train_sequences_2 = pad_sequences(train_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)
test_sequences_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)
test_sequences_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)

In [14]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)

In [15]:
embedding_matrix.shape

(137043, 300)

In [43]:
embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [91]:
question1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
question2 = Input(shape=(MAX_SEQUENCE_LENGTH,))

q1 = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)(question1)
q1 = TimeDistributed(Dense(EMBEDDING_DIM, activation='relu'))(q1)
q1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, ))(q1)

q2 = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)(question2)
q2 = TimeDistributed(Dense(EMBEDDING_DIM, activation='relu'))(q2)
q2 = Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, ))(q2)
merged = concatenate([q1,q2])
merged = Dense(200, activation='relu')(merged)
merged = Dropout(0.1)(merged)
merged = Dense(150, activation='relu')(merged)
merged = Dropout(0.1)(merged)
merged = BatchNormalization()(merged)
merged = Dense(100, activation='relu')(merged)
merged = Dropout(0.1)(merged)
#merged = BatchNormalization()(merged)
#merged = Dense(200, activation='relu')(merged)
#merged = Dropout(DROPOUT)(merged)
#merged = BatchNormalization()(merged)

is_duplicate = Dense(1, activation='sigmoid')(merged)
model = Model(inputs=[question1,question2], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

In [92]:
t0 = time.time()
model_checkpoint = ModelCheckpoint('best_nn1.h5', save_best_only=True, save_weights_only=True)
model.fit([tr1,tr2],
          y_tr.values, epochs=100,
          batch_size=10000, shuffle=True,
          validation_data=([val1,val2], y_val.values), callbacks=[model_checkpoint])
t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Train on 304289 samples, validate on 100000 samples
Epoch 1/100
304289/304289 [==============================] - 322s - loss: 0.5973 - acc: 0.6836 - val_loss: 0.6465 - val_acc: 0.7043

KeyboardInterrupt: 

In [ ]:
17

In [71]:
tr=pd.concat([pd.DataFrame(tr1),pd.DataFrame(tr2)], axis=1).values

In [72]:
val=pd.concat([pd.DataFrame(val1),pd.DataFrame(val2)], axis=1).values

In [17]:
t0 = time.time()

for i in range(10000000):
    t = 10000
t1 = time.time()
#print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Minutes elapsed: 0.010123


In [25]:
train_sequences_2.shape
y_train.shape

(404290, 1)

In [17]:
from keras.layers.merge import concatenate
lstm_layer = LSTM(200, dropout=0.2, recurrent_dropout=0.2)

sequence_1_input = Input(shape=(30,), dtype='int32')
embedded_sequences_1 = embedding_layer(sequence_1_input)
x1 = lstm_layer(embedded_sequences_1)

sequence_2_input = Input(shape=(30,), dtype='int32')
embedded_sequences_2 = embedding_layer(sequence_2_input)
x2 = lstm_layer(embedded_sequences_2)

merged = concatenate([x1, x2])
merged = BatchNormalization()(merged)

merged = Dense(100, activation='relu')(merged)
merged = BatchNormalization()(merged)

merged = Dropout(0.1)(merged)
preds = Dense(1, activation='sigmoid')(merged)


In [18]:
model = Model(inputs=[sequence_1_input, sequence_2_input], \
        outputs=preds)
model.compile(loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['acc'])

model_checkpoint = ModelCheckpoint('best.h5', save_best_only=True, save_weights_only=True)


In [87]:
tr1 = train_sequences_1[100001:]
tr2 = train_sequences_2[100001:]
val1 = train_sequences_1[:100000]
val2 = train_sequences_2[:100000]
y_tr = y_train[100001:]
y_val = y_train[:100000]

In [ ]:
model.fit([tr1, tr2],
          y_tr.values, epochs=20,
          batch_size=10000, shuffle=True,
          validation_data=([val1, val2], y_val.values), callbacks=[model_checkpoint])

Train on 404290 samples, validate on 1000 samples
Epoch 1/20
404290/404290 [==============================] - 1097s - loss: 0.5195 - acc: 0.7374 - val_loss: 0.6357 - val_acc: 0.6210

In [26]:
model

In [50]:

#for itr, ite in folds:
#    ypred = []
len(y_train.ix[itr])

323431

In [73]:
len(folds[1:])

4

In [74]:
#pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]
nu=2
for itr, ite in folds[1:]:
    ypred = []
    for i in range(1):
        from keras.layers.merge import concatenate
        lstm_layer = LSTM(200, dropout=0.2, recurrent_dropout=0.2)

        sequence_1_input = Input(shape=(30,), dtype='int32')
        embedded_sequences_1 = embedding_layer(sequence_1_input)
        x1 = lstm_layer(embedded_sequences_1)

        sequence_2_input = Input(shape=(30,), dtype='int32')
        embedded_sequences_2 = embedding_layer(sequence_2_input)
        x2 = lstm_layer(embedded_sequences_2)

        merged = concatenate([x1, x2])
        merged = BatchNormalization()(merged)

        merged = Dense(100, activation='relu')(merged)
        merged = BatchNormalization()(merged)

        merged = Dropout(0.1)(merged)
        preds = Dense(1, activation='sigmoid')(merged)
        
        
        model = Model(inputs=[sequence_1_input, sequence_2_input], \
            outputs=preds)
        model.compile(loss='binary_crossentropy',
            optimizer='nadam',
            metrics=['acc'])

        model_checkpoint = ModelCheckpoint('best'+str(nu)+'.h5', save_best_only=True, save_weights_only=True)
        
        #model.load_weights('best1.h5')
        
        model.fit([train_sequences_1[itr], train_sequences_2[itr]],
                  y_train.ix[itr].values, epochs=20,
                  batch_size=10000, shuffle=True,
                  validation_data=([train_sequences_1[ite], train_sequences_2[ite]], y_train.ix[ite].values),
                  callbacks=[model_checkpoint])
        ypred = model.predict([train_sequences_1[ite], train_sequences_2[ite]])
        
    pred_train[ite] = ypred
    ypred = pd.DataFrame(ypred)
    ypred.to_csv("stacking/nn_21_"+str(nu)+".csv", index=None)
    nu = nu+1
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

Train on 323431 samples, validate on 80859 samples
Epoch 1/20
323431/323431 [==============================] - 1794s - loss: 0.6254 - acc: 0.6665 - val_loss: 0.6376 - val_acc: 0.6308

/home/fao3864/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:45: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


Train on 323432 samples, validate on 80858 samples
Epoch 1/20
323432/323432 [==============================] - 977s - loss: 0.6256 - acc: 0.6673 - val_loss: 0.6392 - val_acc: 0.6308

In [ ]:
pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]
nu=1
for itr, ite in folds:
    ypred = []
    for i in range(1):
        from keras.layers.merge import concatenate
        lstm_layer = LSTM(200, dropout=0.2, recurrent_dropout=0.2)

        sequence_1_input = Input(shape=(30,), dtype='int32')
        embedded_sequences_1 = embedding_layer(sequence_1_input)
        x1 = lstm_layer(embedded_sequences_1)

        sequence_2_input = Input(shape=(30,), dtype='int32')
        embedded_sequences_2 = embedding_layer(sequence_2_input)
        x2 = lstm_layer(embedded_sequences_2)

        merged = concatenate([x1, x2])
        merged = BatchNormalization()(merged)

        merged = Dense(100, activation='relu')(merged)
        merged = BatchNormalization()(merged)

        merged = Dropout(0.1)(merged)
        preds = Dense(1, activation='sigmoid')(merged)
        
        
        model = Model(inputs=[sequence_1_input, sequence_2_input], \
            outputs=preds)
        model.compile(loss='binary_crossentropy',
            optimizer='nadam',
            metrics=['acc'])

        model_checkpoint = ModelCheckpoint('best'+str(nu)+'_.h5', save_best_only=True, save_weights_only=True)
        
        model.load_weights('best'+str(nu)+'.h5')
        
        model.fit([train_sequences_1[itr], train_sequences_2[itr]],
                  y_train.ix[itr].values, epochs=3,
                  batch_size=10000, shuffle=True,
                  validation_data=([train_sequences_1[ite], train_sequences_2[ite]], y_train.ix[ite].values),
                  callbacks=[model_checkpoint])
        ypred = model.predict([train_sequences_1[ite], train_sequences_2[ite]])
        
    pred_train[ite] = ypred
    ypred = pd.DataFrame(ypred)
    #ypred.to_csv("stacking/nn_21_"+str(nu)+".csv", index=None)
    nu = nu+1
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

Train on 323431 samples, validate on 80859 samples
Epoch 1/3
323431/323431 [==============================] - 1092s - loss: 0.3319 - acc: 0.8493 - val_loss: 0.3811 - val_acc: 0.8222

/home/fao3864/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:45: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


Train on 323431 samples, validate on 80859 samples
Epoch 1/3
323431/323431 [==============================] - 1088s - loss: 0.3238 - acc: 0.8539 - val_loss: 0.3906 - val_acc: 0.8161

In [25]:
preds_train

1

In [24]:
len(X_train)

404290

In [ ]:
0.3595 0.4362

In [39]:
import gc
gc.collect()

1453

In [26]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['lstm1']

In [31]:
pred_train[pred_train.lstm1<0.00000001]

,lstm1
1772,7.286283e-09
74958,1.004034e-09
124821,8.097936e-09
162351,2.967399e-09
167273,1.296422e-09
218027,4.586146e-09
225416,6.235059e-09
240480,2.459652e-09
341020,8.912590e-09


In [32]:
pred_train.to_csv("stacking/nn_21_lstm1.csv", index=None)

In [35]:
        from keras.layers.merge import concatenate
        lstm_layer = LSTM(200, dropout=0.2, recurrent_dropout=0.2)

        sequence_1_input = Input(shape=(30,), dtype='int32')
        embedded_sequences_1 = embedding_layer(sequence_1_input)
        x1 = lstm_layer(embedded_sequences_1)

        sequence_2_input = Input(shape=(30,), dtype='int32')
        embedded_sequences_2 = embedding_layer(sequence_2_input)
        x2 = lstm_layer(embedded_sequences_2)

        merged = concatenate([x1, x2])
        merged = BatchNormalization()(merged)

        merged = Dense(100, activation='relu')(merged)
        merged = BatchNormalization()(merged)

        merged = Dropout(0.1)(merged)
        preds = Dense(1, activation='sigmoid')(merged)
        
        
        model = Model(inputs=[sequence_1_input, sequence_2_input], \
            outputs=preds)
        model.compile(loss='binary_crossentropy',
            optimizer='nadam',
            metrics=['acc'])

        model_checkpoint = ModelCheckpoint('best_.h5', save_best_only=True, save_weights_only=True)
        
        model.load_weights('best.h5')
        
        model.fit([train_sequences_1[:], train_sequences_2[:]],
                  y_train.ix[:].values, epochs=3,
                  batch_size=10000, shuffle=True,
                  validation_data=([train_sequences_1[:], train_sequences_2[:]], y_train.ix[:].values),
                  callbacks=[model_checkpoint])

Train on 404290 samples, validate on 404290 samples
Epoch 1/3
404290/404290 [==============================] - 1809s - loss: 0.3182 - acc: 0.8565 - val_loss: 0.2784 - val_acc: 0.8837

In [ ]:
        ypred = model.predict([train_sequences_1[ite], train_sequences_2[:]])

In [82]:
model.load_weights('best1.h5')

In [36]:
test_pred = model.predict([test_sequences_1, test_sequences_2])

In [84]:
test_pred.shape

(2345796, 1)

In [85]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [88]:
test_pred = foo1(test_pred)

In [90]:
test_pred[:5]

array([[ 0.05790832],
       [ 0.42116681],
       [ 0.59141481],
       [ 0.00314641],
       [ 0.72020227]], dtype=float32)

In [37]:
test_pred = pd.DataFrame(test_pred)
test_pred.columns = ['lstm1']

In [38]:
test_pred.to_csv("stacking/nn_21_lstm1_test.csv", index=None)

In [24]:
submit = pd.read_csv("data/sample_submission.csv")

In [25]:
submit['is_duplicate'] = test_pred

In [27]:
submit.to_csv("stacking/nn_21_test.csv", index=None)